좋아요+댓글달기 모듈 파일

txt파일 내용으로 댓글달기

In [ ]:
# 브라우저를 제어하기 위해 selenium 모듈의 webdriver를 import
from selenium import webdriver

# 브라우저를 옵션을 이용해서 활성화시키기 위해 Options를 import
from selenium.webdriver.chrome.options import Options

# 클릭과 캡쳐를 위해 keys를 import
from selenium.webdriver.common.keys import Keys

# 작업과 작업 사이에 딜레이를 주기 위해 time 모듈을 import
import time

# 댓글 리스트 중 무작위로 뽑아오기 위해 random 모듈을 import
import random

class ReplyBot:
    def __init__(self, replyfile):        
        # 홈페이지를 변수에 저장
        self.querry = "https://www.instagram.com/explore/tags/"
        
        # 셀레늄 웹드라이버에 입력할 옵션을 지정
        self.options = Options()
        
        # 옵션에 해상도를 입력
        self.options.add_argument("--window-size=1920, 1080")
        
        # 화면이 존재하지 않는 서버에서 사용한다면 해상도 입력 옵션 사용 X
        # ex) 리눅스 cli 서버, 우분투 서버
        # 아래 headless 옵션을 사용
        # self.options.add_argument("headless")
        
        # 크롬 드라이버 실행
        self.driver = webdriver.Chrome(
            executable_path="chromedriver_win32/chromedriver.exe",
            chrome_options=self.options)
        
        # 불러들인 replyfile을 utf-8 버전으로 불러옴
        # w,r,a 옵션을 안써줄 경우 디폴트값인 r로 실행됨
        self.replyfile = open(replyfile, encoding = "utf-8")
        
        # 댓글 내용 리스트를 제작
        self.replylist = self.replyfile.read().split("\n")
        
    # 크롤러 종료 메서드
    def kill(self):
        self.driver.quit()
        
    # 스크린샷 메서드
    # html 요소를 가지고 스크린샷을 하는 것이 아닌 전체를 스크린샷
    def save_screenshot(self, filename):
        self.driver.save_screenshot(filename)
        
    # 인스타그램 로그인 메서드
    def login(self, id, ps):
        
        # 로그인 페이지로 이동
        self.driver.get("https://www.instagram.com/accounts/login/")
        
        # 다음 동작을 위해 딜레이 시간 부여
        time.sleep(5)
        
        # 아이디와 패스워드 입력을 위해 <input> 태그 찾기
        input_field = self.driver.find_elements_by_tag_name("input")
        
        # 첫 번째 요소 아이디를 입력
        input_field[0].send_keys(id)
        
        # 두 번째 요소 패스워드를 입력
        input_field[1].send_keys(ps)
        
        # 엔터키를 눌러 로그인 마무리
        input_field[1].send_keys(Keys.RETURN)
        
        # 다음 동작을 위해 딜레이 시간 부여
        time.sleep(5)
        
    # 인스타그램 태그 검색 메서드
    def search_tag(self, tag):
        
        # 위에서 지정한 URL 주소와 태그를 결합해서 브라우저를 열어줌
        self.driver.get(self.querry + tag)
        
        # 다음 동작을 위해 딜레이 시간 부여
        time.sleep(5)
        
    # 최근 게시물 첫 번째 사진을 선택하여 클릭하는 메서드
    def select_picture(self):
        
        # 최근 게시물의 위치를 recent_picture_xpath에 지정
        recent_picture_xpath = '//*[@id="react-root"]/section/main/article/div[2]/div/div[1]/div[1]'
        
        # recent_picture_xpath의 요소를 가져와서 저장
        recent_picture = self.driver.find_element_by_xpath(recent_picture_xpath)
        
        # 최근 게시물 클릭
        recent_picture.click()
        
        # 다음 동작을 위해 딜레이 시간 부여
        time.sleep(5)
        
    # 댓글을 남기는 메서드
    def send_reply(self, text):
        textarea = self.driver.find_element_by_tag_name("textarea")
        textarea.send_keys(text + Keys.RETURN)
        
        
    # 게시물들을 돌아다니면서 '좋아요' 버튼을 누르고 댓글도 작성하는 메서드
    # num 변수에 몇 번 반복할 것인지를 저장
    # -1 을 입력하면 직접 종료하기 전까지 무한정으로 계속 '좋아요' 버튼 클릭
    def press_like_and_reply(self, num):
        
        # 반복할 횟수를 count에 저장
        count = num
        
        # count가 0이 될때까지 계속 반복
        # num에 -1이 들어오게 되면 무한 반복
        while count != 0:
                        
            # "좋아요"가 포함된 태그를 기반으로 찾기
            # <svg> 태그를 가진 모든 요소를 저장
            like_button = self.driver.find_elements_by_tag_name("svg")
            
            # <textarea> 태그를 가진 요소를 저장
            reply = self.driver.find_elements_by_tag_name("textarea")
            
            # <svg> 태그 내부에 aria-label이라는 어트리뷰트(속성)를 가지고 "좋아요"를 가진 경우에만 클릭
            # "좋아요 취소" 일 경우에는 클릭 X

            list_like = []
            for ele in like_button:
                if ele.get_attribute("aria-label") == "좋아요":
                    list_like.append(ele)

            list_like[1].click() # 1번 인덱스를 클릭하는 이유 : 0번다음 1번이 좋아요 버튼 여기까지가 좋아요 누르기
            
            # 여기서부터 댓글달기
            # send_reply 메서드를 호출하여
            try:
                self.send_reply(random.choice(self.replylist))
                
            except:
                time.sleep(5)
                try:
                    self.send_reply(random.choice(self.replylist))
                except:                                      
                    time.sleep(5)
                    self.send_reply(random.choice(self.replylist))
            
        
            # 다음 게시물로 넘어감
            next_button = '/html/body/div[6]/div[2]/div/div[2]/button/div/span'            
            next_button_element = self.driver.find_element_by_xpath(next_button)
            next_button_element.click()

            count -= 1
            
            # 다음 동작을 위해 딜레이 시간 부여
            time.sleep(5) 

메인 파일

In [ ]:
# 인자를 받기 위해 sys 모듈 import
import sys

# 디렉토리를 만들기 위해 os 모듈 import
import os

# insta_bot_like 모듈 import
import insta_bot_reply as ib

# 아이디를 입력
id = sys.argv[1]

# 패스워드를 입력
ps = sys.argv[2]

# 태그 입력
tag = sys.argv[3]

# 입력할 댓글이 저장된 파일을 입력
replyfile = sys.argv[4]
  
# 반복할 횟수 입력
NUMBER = int(sys.argv[5].strip())

# 크롤러 호출
BOT = ib.ReplyBot(replyfile)

# 인스타에 로그인
BOT.login(id, ps)

# 태그 검색
BOT.search_tag(tag)
        
# 사진 첫 장을 선택하고
BOT.select_picture()
        
# 좋아요를 하면서 댓글을 입력하고 한 장씩 넘기기
BOT.press_like_and_reply(NUMBER)

# 하나로 묶은 매크로 사용
# BOT.insta_jungbok(tag, NUMBER)

# 작업 완료 후 크롤러 종료
BOT.kill()

# 스크레이핑
## 책 : 파이썬을 이용한 웹 크롤링과 스크레이핑

- 크롤링 : 스스로 웹 페이지를 탐색, 홈페이지 내에서 콘텐츠를 추출
- 스크레이핑 : 해당 페이지에서 특정 정보를 추출(소스 코드 등)
범위 : 크롤링 > 스크레이핑

urllib
- URL 작업을 위한 데이터 수집 패키지
 - urllib.request : URL을 열고 웹페이지를 추출
 - urllib.error : urllib.request 에서 발생한 에러
 - urllib.parse : URL 구문 분석

urllib을 이용한 웹 페이지 추출

In [3]:
from urllib.request import urlopen

In [4]:
f = urlopen("http://hanbit.co.kr")

In [5]:
print(f)

- 파일 입출력과 비슷(객체를 이용하여 open)

In [4]:
print(type(f))

<class 'http.client.HTTPResponse'>


http://hanbit.co.kr 의 소스코드

In [5]:
print(f.read())

b'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>\xed\x95\x9c\xeb\xb9\x9b\xec\xb6\x9c\xed\x8c\x90\xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="\xed\x95\x9c\xeb\xb9\x9b\xec\xb6\x9c\xed\x8c\x90\xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac"/>\r\n<meta property="og:description" content="\xeb\x8d\x94 \xeb\x84\x93\xec\x9d\x80 \xec\x84\xb8\xec\x83\x81, \xeb\x8d\x94 \xeb\x82\x98\xec\x9d\x80 \xeb\xaf\xb8\xeb\x9e\x98\xeb\xa5\xbc \xec\x9c\x84\xed\x95\x9c \xec\x95\x84\xec\x8b\x9c\xec\x95\x84 \xec\xb6\x9c\xed\x8c\x90 \xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac :: \xed\x95\x9c\xeb\xb9\x9b\xeb\xaf\

웹 페이지의 상태 확인

In [7]:
f.status

200

- 200은 정상 동작 중이라는 뜻
- 404 : not found - 해당 페이지가 없음

특정 타입에 맞는 헤더 출력

In [13]:
print(f.getheader('Content-Type'))

text/html; charset=UTF-8


- 인코딩 : 코드화
- 디코딩 : 역코드화(사람이 이해할 수 있도록 바꿔주는 것)
 - 디코딩을 하기 위해서는 인코딩이 어떻게 되어있는지 확인해야함

page 53
<br>디코딩 하는 방법

In [7]:
f.info()

In [18]:
# 표준 에러를 지정하기 위해 sys 모듈을 import
import sys

# 웹 페이지 추츨을 위해 urlopen 모듈을 import
from urllib.request import urlopen

# 해당 홈페이지를 객체로 저장
f = urlopen("http://hanbit.co.kr/store/books/full_book_list.html")

# 객체 f의 정보를 기반으로 인코딩 방식을 추출하여 encoding 객체에 저장
encoding2 = f.info().get_content_charset(failobj = "utf-8")

# 인코딩 방식을 출력, 출력 과정에서 생긴 에러를 file 객체에 저장 후 출력
# stderr로 지정해서 출력
# 아래 print 구문이 에러로 출력
print('encoding :', encoding2, file=sys.stderr) # file=sys.stderr는 print 함수의 형식

# 추출한 인코딩 방식으로 디코딩
text = f.read().decode(encoding2)
print(text)


<!DOCTYPE html>
<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>한빛출판네트워크</title>
<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> 
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta property="og:type" content="website"/>
<meta property="og:title" content="한빛출판네트워크"/>
<meta property="og:description" content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀"/>
<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />
<meta property="og:url" content="https://hanbit.co.kr/store/books/full_book_list.html"/>
<link rel="canonical" href="https://hanbit.co.kr/store/books/full_book_list.html" />
<meta name="keywords" content="한빛미디어,한빛아카데미,한빛비즈,한빛라이프,한빛에듀,리얼타임,대관서비스,책,출판사,IT전문서,IT활용서,대학교재,경제경영,어린이/유아,실용/여행,전자책,자격증,교육,세미나,강의,ebook,정보교과서" />
<meta name="description" content="더 넓은 세상, 더 나은 

encoding : utf-8


- 위 코드를 복사한후 urlopen_encoding.py 파일을 만든 후
- cmd창에 python urlopen_encoding.py > dp.html 입력하면 dp.html 파일이 생성됨

page54 : meta 태그에서 인코딩 방식 추출
- \<meta charset='utf-8'/>